<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/text2cite_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!nvidia-smi

Mon Dec 23 01:19:10 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install langid
!pip install tqdm

     |████████████████████████████████| 1.9MB 2.7MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp36-none-any.whl size=1941190 sha256=70328874a5006aa77e89e43843bedef8ad08b3ee3682a90a08ab40e772347401
  Stored in directory: /root/.cache/pip/wheels/29/bc/61/50a93be85d1afe9436c3dc61f38da8ad7b637a38af4824e86e
Successfully built langid


In [0]:
import zipfile
import os
import sys
import pandas as pd
import numpy as np
import gc
from urllib import request
import json
import itertools
import gzip
import shutil
import ast
import pickle 
import nltk
from langid.langid import LanguageIdentifier, model
from tqdm import tqdm

# Necessary Functions

In [0]:
# Function to unip a file from semantic scholar to a json file, then open that json file
def dlandopen(url, file, json ):
    request.urlretrieve(url, file )
    with gzip.open(file, 'rb') as f_in:
        with open(json, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(file)
    f = open(json)
    mylistH = f.readlines()
    del f
    return mylistH

In [0]:
# This is probably not the best way to go about it, but this is the code to convert the opened file into a pandas dataframe
#Akul, it may be worth figuring out if you can make a dataframe directly, without going line-by-line 
def firstDataframe(mylist, start, finish):
    data = []

    for line in itertools.islice(mylist , start , finish):
       data.append(json.loads(line))

    df = pd.DataFrame(data)
    del data
    gc.collect()
    return df

In [0]:
def lowerCaseElements(list):
  newList = []
  for i in list:
    newList.append(i.lower())
  return newList

In [0]:
def medicalRefinement(dftbr):
  dftbr['strsources'] = dftbr['sources'].astype('str')
  dftbr['totalCites2'] = dftbr['outCitations'] + dftbr['inCitations'] 
  dftbr = dftbr[dftbr['totalCites2'].str.len() >= 1]  #outCitations - all papers where papers cited is greater than 1
  dftbr = dftbr[ (dftbr.pmid != '' ) | (dftbr.strsources.str.contains("Medline"))]
  dftbr.drop(['journalVolume','journalPages','year','authors','sources','doiUrl', 'strsources'], axis = 1)
  return dftbr

In [0]:
# This is a code to filter out some of the non cs-papers, or papers with no references or citations
def dfRefinement(dftbr):
    dftbr['Sources3'] = dftbr['sources'].astype('str') 
    dftbr = dftbr[~dftbr.Sources3.str.contains("Medline") ]  # Remove all medline papers
    
    dftbr =  dftbr[ ( dftbr.astype(str)['outCitations'] != '[]' ) | ( dftbr.astype(str)['inCitations'] != '[]' ) ] #remove any paper with no citations
    dftbr['totalCites2'] = dftbr['outCitations'] + dftbr['inCitations'] 
    dftbr = dftbr[ dftbr['totalCites2'].str.len() >= 1] #make sure papers have at least one citation. 

    #Started to filter out non math/computer science/physics/EE papers . 

    #Akul, this is probably what you'll be spending the most time on; creating more filters. 

    # Tip, Excel is create for creating a bunch lines with variations
    # https://docs.google.com/spreadsheets/d/129lf-AhKCGJKucpMXCxN_0HhXUw41KYTt3KIVS0u1Dc/edit?usp=sharing
    
    dftbr.drop(['journalVolume','journalPages','year','authors','sources','doiUrl', 'Sources3'], axis = 1)

    dftbr[dftbr.pmid == '']

    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("economics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharmacology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("plant") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nutrition") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("medical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("molecular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cancer") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("surgical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("french") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("political") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("modernism") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("jewish") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("wildlife") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("women's studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("philosophy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("english studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("business") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("agriculture") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("korean studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("zoological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("song-yuan studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("phonetics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("physiotherapy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("geochronometria") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hispanic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pediatric") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psychology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("feminist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("archaeology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hepatitis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gynecologic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("philology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("paleontology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gender") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ornithology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dental") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurocytology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("petrology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("film") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("history") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("multihandicapped") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("theatre") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuroradiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psycholinguistic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biotelemetry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hernia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("horticulture") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("photosynthetica") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biological theory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("forestry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("second language") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("seismology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("schizophrenia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("formal logic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("drug") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemists") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("prehistory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("happiness studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gambling") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("economic journal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("food") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("carbonates") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("degradation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("financial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chaucer") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("naturalist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("management control") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("language learning") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("great plains") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("religion") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("geological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("banking") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("jewry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("acoustics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("clinical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("botany") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("mammal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("animal science") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anthropological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hydrobiological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pain management") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("crime") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("social change") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("african studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fertilizer") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biomed") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cardiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("geotectonics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sports") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("archaeological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("radioanalytical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pedagogy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ornithologists") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biomass") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gynecology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("marine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biosphere") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("transplantation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("wood") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("oncology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("particle") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("depression") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ultrasonic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gastroenterology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("diversity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("agri ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lipid ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("protein ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("xeno") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nuclear") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("criminology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("metabolism") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("forensic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("heredity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("conservation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("atmospheric") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("earth") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("geo") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("aquatic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("husbandry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("rangifier") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("weather") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("potato") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("plasma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("veterinary") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("amphibian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("elife") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biometrics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("insect") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("endoscopic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ecosystem") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antiviral") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("armed") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gastroprotective") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("microwave") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("malaysia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("t cells") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("monoclonal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("angiotensin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gastrointestinal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("parasites") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("unemployment") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("television") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("soil") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("government") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cultural") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("culture") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("obesity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("uganda") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("b cell") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antioxidant") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("elastin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("collagen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("asia-pacific") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antibacterial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("juice") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lactobacillus") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pelvic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biodegradation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("crude oil") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sediment") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("childbirth") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunoglobulin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("myeloma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("infection") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cardiovascular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("adrenoceptor") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bodyweight") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("squat") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("deadlift") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cardiomyoplasty") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cardio") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("phallus") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("intravesical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("exstrophy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cloacal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fermentation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anger") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("aggression") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("china") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("earlobe") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("estrogen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("atrazine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hydroxytamoxifen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("proliferation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("liver") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cell culture") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("megafauna") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("smoking") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("childhood") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("overwei") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("latissmus") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("myocutaneous") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cervicof") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("selenium") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("maritime") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("agricultural") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sudanese") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("refugee") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("wine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("glycine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("betaine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurodegeneration") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("diabetes") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nitrates") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("myeloid") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("salinity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hindi") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bengali") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("wound") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("postoperative") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nucleus") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("thylakoid") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("celiac") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("disease") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("middle school") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("amino acid") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bacterial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biofilm") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("larvae") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("naval") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("asparaginases") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cataract") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hepatoxic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("toxic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("vitamin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lumbar") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("viral") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("melatonin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("in vitro ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("in vivo") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("b-cell") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("t-cell") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("interleukin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("zinc") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fluoride") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("stannous") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("macrophage") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antigen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("breast") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("methotrexate") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("perilesional") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("organic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("phosphates") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("organ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("vaccine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharmaceutical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nutrients") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("umbilical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neonate") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biomarker") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sailboat") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("drinking age") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("scrotal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("extrasosseous") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chondroma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("saudi arabia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("embryonic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("saccaharification") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("genetic markers") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("urine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("contaminated") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ground water") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("groundwater") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("outdoor") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("schizo") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hepatocellular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("carcinoma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antibodies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psoriasis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hydroxychloro") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("acrosomal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("mammogram") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("disinfectant") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("germicide") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("tyrosine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("kinase") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("inhibitors") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hemostasis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pollution") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pregnancy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immune") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("salmonella") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("shanghai") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immigrant") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pathogen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ethylene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("phenology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("binge eating") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("xanthan") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ribosomal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chromosomal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("oligonucleotide") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chromosome") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("angiogenesis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bipolar") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("disorder") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("atorvastatin") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("embryo") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytotoxic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("papillomaviruses") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytometry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cervical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cyclophosphamide") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("acupuncture") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nigerian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pancreatitis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pancrea") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("tract infection") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("inflammatory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dermatoglyphic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("corticosteroid") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("xerostomia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuropsychological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("glycemic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurotrophic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("carotene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("prognosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ozone") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("drunk") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("edible") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("proteomic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("mitochondrial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("malignant") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lymphoma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("l-serine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("protease") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("leukocytes") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("phagocytosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sickle") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sexual") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("paternity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("maternity") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemotherapy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharmacogenomics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("mucoadhesive") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("soccer players") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cell membrane") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("benzoyl") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bronchial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gastrointestintal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immune deficiency") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("infectious") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anticoagulate") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sociomaterial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("amsterdam") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("earthquake") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("kidney") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ligniolytic enzyme") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("femur") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("angiogenes") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("serum") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("protein expression") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("health effect") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("carcinogen") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cysteine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharmacoepdemiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("adenine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("guanine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytosene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("thymine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("althlete") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bioavailability") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("green tea") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dentistry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuromuscular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hydroxypropyl") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fibularis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cadaveric") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pickles") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("florescence") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ribosome") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("antidiorrhoeal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chimpanzee") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("newborn") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anesthesiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dermatology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anesthesiolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dermatolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("radiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("radiolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("emergency medicine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("genetics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nuclear medicine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("obstetrics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gynelcolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ophthalmolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("patholog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("rehabilitation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("rehabilitate") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psychiatr") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("oncolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("surgeon") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("urology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytotechnologist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytotechnology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("electroneuro") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("kinesiolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("kinesiotherap") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lactation") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("therapist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("therapy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("perfusion") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("physiotherap") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("polysomnograph") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ambulatory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("audiolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dietitian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dietetic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("therapeutic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ophthalmic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("optometrist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("optometry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("optometr") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("orthopy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("orthopist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("orthop") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("optician") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("dialysis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("electrocardiograph") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("environmental health") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemiologist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemiolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("histolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nurse") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nursery") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("midwife") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("osteopathy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pharmac") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("allopathic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("podiatr") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sonograph") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("veterinar") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("x-ray") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("asthma") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("autism") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("autistic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("influenza") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("avian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("birth defect") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chlamydia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fatigue") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epilepsy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fetal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("herpes") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("genital") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gonorrhea") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("heart disease") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pulmonary") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("traumatic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("zika") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("tuberculosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("scabies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("microcephaly") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("middle east") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("papillomavirus") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("giardiasis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("aesthetic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epistemolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("metaphysic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ageing") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("human affairs") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("victorian studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("estuaries") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("estuary") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("fungal") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("german studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuroscience") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("utopian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("infants") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("young children") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("aging") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cellulose") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("labor") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("labour") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("vaccines") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("israel") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("lung cancer") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunohematology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("digestive") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunopathology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("radicalism") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("radical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("caffeine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("unicorn") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("orthodontics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("orthodont") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immuno") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("eukaryotic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nicotine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("tobacco") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("forest science") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("acoustical") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("acoustic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cytogenetic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("peptidomics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("virology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("adipocyte") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cellular") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("animal learning") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("social welfare") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ophthalmology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("rheumatism") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nutrafoods") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("morphology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("tissue") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("rhinology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("zoomorphology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biorheology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("nanotoxicology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hygiene") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("spectrometry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("shellfish") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("analgesia") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bioorganic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("sociolinguistic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("photogrammetry") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemosphere") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("raptor research") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("meridian studies") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("drugs") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("angiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("pancreas") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("craniofacial") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bioenergetics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bioenergetic") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("traumatology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("periodontology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("methodology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("traumat") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemi") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("globalization") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("electrophysiology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("cardiac ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("humanit ") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("infections") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("reproductive") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("atherosclerosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("gerontology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("midwestern") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("bioethics") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("myology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("epidemiological") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("immunology") ]    
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biolog") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("chemi") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hypertension") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("primatology") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("management") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("metals") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("water") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("hindu") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("muslim") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("buddhist") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("missouri") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("peptides") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("clima") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("education") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("behavioral") ]
    dftbr = dftbr[~dftbr.journalName.str.lower().str.contains("biotech") ]

    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("economics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharmacology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("plant") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nutrition") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("medical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("molecular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cancer") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("surgical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("french") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("political") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("modernism") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("jewish") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("wildlife") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("women's studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("philosophy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("english studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("business") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("agriculture") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("korean studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("zoological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("song-yuan studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("phonetics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("physiotherapy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("geochronometria") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hispanic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pediatric") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psychology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("feminist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("archaeology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hepatitis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gynecologic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("philology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("paleontology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gender") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ornithology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dental") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurocytology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("petrology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("film") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("history") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("multihandicapped") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("theatre") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuroradiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psycholinguistic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biotelemetry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hernia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("horticulture") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("photosynthetica") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biological theory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("forestry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("second language") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("seismology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("schizophrenia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("formal logic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("drug") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemists") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("prehistory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("happiness studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gambling") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("economic journal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("food") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("carbonates") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("degradation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("financial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chaucer") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("naturalist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("management control") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("language learning") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("great plains") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("religion") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("geological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("banking") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("jewry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("acoustics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("clinical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("botany") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("mammal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("animal science") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anthropological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hydrobiological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pain management") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("crime") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("social change") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("african studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fertilizer") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biomed") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cardiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("geotectonics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sports") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("archaeological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("radioanalytical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pedagogy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ornithologists") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biomass") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gynecology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("marine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biosphere") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("transplantation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("wood") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("oncology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("particle") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("depression") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ultrasonic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gastroenterology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("diversity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("agri ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lipid ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("protein ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("xeno") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nuclear") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("criminology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("metabolism") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("forensic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("heredity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("conservation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("atmospheric") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("earth") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("geo") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("aquatic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("husbandry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("rangifier") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("weather") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("potato") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("plasma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("veterinary") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("amphibian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("elife") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biometrics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("insect") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("endoscopic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ecosystem") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antiviral") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("armed") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gastroprotective") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("microwave") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("malaysia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("t cells") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("monoclonal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("angiotensin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gastrointestinal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("parasites") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("unemployment") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("television") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("soil") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("government") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cultural") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("culture") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("obesity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("uganda") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("b cell") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antioxidant") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("elastin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("collagen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("asia-pacific") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antibacterial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("juice") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lactobacillus") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pelvic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biodegradation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("crude oil") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sediment") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("childbirth") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunoglobulin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("myeloma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("infection") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cardiovascular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("adrenoceptor") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bodyweight") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("squat") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("deadlift") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cardiomyoplasty") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cardio") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("phallus") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("intravesical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("exstrophy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cloacal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fermentation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anger") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("aggression") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("china") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("earlobe") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("estrogen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("atrazine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hydroxytamoxifen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("proliferation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("liver") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cell culture") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("megafauna") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("smoking") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("childhood") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("overwei") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("latissmus") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("myocutaneous") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cervicof") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("selenium") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("maritime") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("agricultural") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sudanese") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("refugee") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("wine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("glycine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("betaine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurodegeneration") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("diabetes") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nitrates") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("myeloid") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("salinity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hindi") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bengali") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("wound") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("postoperative") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nucleus") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("thylakoid") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("celiac") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("disease") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("middle school") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("amino acid") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bacterial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biofilm") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("larvae") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("naval") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("asparaginases") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cataract") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hepatoxic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("toxic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("vitamin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lumbar") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("viral") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("melatonin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("in vitro ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("in vivo") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("b-cell") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("t-cell") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("interleukin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("zinc") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fluoride") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("stannous") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("macrophage") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antigen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("breast") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("methotrexate") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("perilesional") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("organic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("phosphates") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("organ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("vaccine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharmaceutical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nutrients") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("umbilical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neonate") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biomarker") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sailboat") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("drinking age") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("scrotal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("extrasosseous") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chondroma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("saudi arabia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("embryonic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("saccaharification") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("genetic markers") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("urine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("contaminated") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ground water") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("groundwater") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("outdoor") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("schizo") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hepatocellular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("carcinoma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antibodies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psoriasis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hydroxychloro") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("acrosomal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("mammogram") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("disinfectant") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("germicide") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("tyrosine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("kinase") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("inhibitors") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hemostasis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pollution") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pregnancy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immune") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("salmonella") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("shanghai") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immigrant") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pathogen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ethylene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("phenology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("binge eating") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("xanthan") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ribosomal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chromosomal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("oligonucleotide") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chromosome") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("angiogenesis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bipolar") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("disorder") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("atorvastatin") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("embryo") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytotoxic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("papillomaviruses") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytometry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cervical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cyclophosphamide") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("acupuncture") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nigerian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pancreatitis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pancrea") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("tract infection") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("inflammatory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dermatoglyphic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("corticosteroid") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("xerostomia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuropsychological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("glycemic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurotrophic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("carotene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("prognosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ozone") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("drunk") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("edible") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("proteomic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("mitochondrial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("malignant") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lymphoma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("l-serine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("protease") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("leukocytes") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("phagocytosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sickle") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sexual") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("paternity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("maternity") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemotherapy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharmacogenomics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("mucoadhesive") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("soccer players") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cell membrane") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("benzoyl") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bronchial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gastrointestintal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immune deficiency") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("infectious") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anticoagulate") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sociomaterial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("amsterdam") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("earthquake") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("kidney") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ligniolytic enzyme") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("femur") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("angiogenes") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("serum") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("protein expression") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("health effect") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("carcinogen") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cysteine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharmacoepdemiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("adenine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("guanine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytosene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("thymine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("althlete") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bioavailability") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("green tea") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dentistry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuromuscular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hydroxypropyl") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fibularis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cadaveric") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pickles") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("florescence") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ribosome") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("antidiorrhoeal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chimpanzee") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("newborn") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anesthesiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dermatology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anesthesiolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dermatolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("radiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("radiolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("emergency medicine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("genetics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nuclear medicine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("obstetrics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gynelcolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ophthalmolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("patholog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("rehabilitation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("rehabilitate") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psychiatr") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("oncolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("surgeon") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("urology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytotechnologist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytotechnology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("electroneuro") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("kinesiolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("kinesiotherap") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lactation") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("therapist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("therapy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("perfusion") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("physiotherap") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("polysomnograph") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ambulatory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("audiolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dietitian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dietetic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("therapeutic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ophthalmic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("optometrist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("optometry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("optometr") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("orthopy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("orthopist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("orthop") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("optician") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("dialysis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("electrocardiograph") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("environmental health") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemiologist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemiolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("histolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nurse") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nursery") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("midwife") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("osteopathy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pharmac") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("allopathic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("podiatr") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sonograph") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("veterinar") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("x-ray") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("asthma") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("autism") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("autistic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("influenza") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("avian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("birth defect") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chlamydia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fatigue") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epilepsy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fetal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("herpes") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("genital") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gonorrhea") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("heart disease") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pulmonary") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("traumatic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("zika") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("tuberculosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("scabies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("microcephaly") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("middle east") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("papillomavirus") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("giardiasis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("aesthetic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epistemolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("metaphysic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ageing") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("human affairs") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("victorian studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("estuaries") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("estuary") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("fungal") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("german studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuroscience") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("utopian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("infants") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("young children") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("aging") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cellulose") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("labor") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("labour") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("vaccines") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("israel") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("lung cancer") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunohematology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("digestive") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunopathology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("radicalism") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("radical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("caffeine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("unicorn") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("orthodontics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("orthodont") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immuno") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("eukaryotic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nicotine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("tobacco") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("forest science") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("acoustical") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("acoustic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cytogenetic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("peptidomics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("virology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("adipocyte") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cellular") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("animal learning") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("social welfare") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ophthalmology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("rheumatism") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nutrafoods") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("morphology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("tissue") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("rhinology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("zoomorphology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biorheology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("nanotoxicology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hygiene") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("spectrometry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("shellfish") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("analgesia") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bioorganic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("sociolinguistic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("photogrammetry") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemosphere") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("raptor research") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("meridian studies") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("drugs") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("angiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("pancreas") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("craniofacial") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bioenergetics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bioenergetic") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("traumatology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("periodontology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("methodology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("traumat") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemi") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("globalization") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("electrophysiology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("cardiac ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("humanit ") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("infections") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("reproductive") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("atherosclerosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("gerontology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("midwestern") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("bioethics") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("myology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("epidemiological") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biolog") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("chemi") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hypertension") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("primatology") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("management") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("metals") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("water") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("hindu") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("muslim") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("buddhist") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("missouri") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("peptides") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("clima") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("education") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("behavioral") ]
    dftbr = dftbr[~dftbr.venue.str.lower().str.contains("biotech") ]

    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("economics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharmacology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("plant") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nutrition") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("medical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("molecular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cancer") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("surgical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("french") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("political") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("modernism") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("jewish") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("wildlife") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("women's studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("philosophy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("english studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("business") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("agriculture") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("korean studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("zoological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("song-yuan studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("phonetics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("physiotherapy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("geochronometria") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hispanic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pediatric") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psychology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("feminist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("archaeology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hepatitis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gynecologic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("philology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("paleontology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gender") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ornithology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dental") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurocytology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("petrology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("film") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("history") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("multihandicapped") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("theatre") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuroradiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psycholinguistic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biotelemetry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hernia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("horticulture") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("photosynthetica") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biological theory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("forestry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("second language") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("seismology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("schizophrenia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("formal logic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("drug") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemists") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("prehistory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("happiness studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gambling") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("economic journal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("food") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("carbonates") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("degradation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("financial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chaucer") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("naturalist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("management control") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("language learning") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("great plains") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("religion") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("geological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("banking") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("jewry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("acoustics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("clinical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("botany") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("mammal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("animal science") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anthropological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hydrobiological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pain management") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("crime") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("social change") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("african studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fertilizer") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biomed") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cardiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("geotectonics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sports") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("archaeological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("radioanalytical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pedagogy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ornithologists") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biomass") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gynecology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("marine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biosphere") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("transplantation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("wood") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("oncology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("particle") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("depression") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ultrasonic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gastroenterology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("diversity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("agri ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lipid ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("protein ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("xeno") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nuclear") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("criminology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("metabolism") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("forensic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("heredity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("conservation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("atmospheric") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("earth") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("geo") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("aquatic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("husbandry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("rangifier") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("weather") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("potato") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("plasma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("veterinary") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("amphibian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("elife") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biometrics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("insect") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("endoscopic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ecosystem") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antiviral") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("armed") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gastroprotective") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("microwave") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("malaysia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("t cells") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("monoclonal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("angiotensin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gastrointestinal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("parasites") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("unemployment") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("television") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("soil") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("government") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cultural") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("culture") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("obesity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("uganda") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("b cell") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antioxidant") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("elastin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("collagen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("asia-pacific") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antibacterial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("juice") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lactobacillus") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pelvic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biodegradation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("crude oil") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sediment") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("childbirth") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunoglobulin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("myeloma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("infection") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cardiovascular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("adrenoceptor") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bodyweight") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("squat") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("deadlift") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cardiomyoplasty") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cardio") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("phallus") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("intravesical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("exstrophy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cloacal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fermentation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anger") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("aggression") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("china") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("earlobe") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("estrogen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("atrazine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hydroxytamoxifen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("proliferation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("liver") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cell culture") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("megafauna") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("smoking") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("childhood") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("overwei") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("latissmus") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("myocutaneous") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cervicof") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("selenium") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("maritime") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("agricultural") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sudanese") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("refugee") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("wine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("glycine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("betaine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurodegeneration") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("diabetes") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nitrates") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("myeloid") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("salinity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hindi") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bengali") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("wound") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("postoperative") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nucleus") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("thylakoid") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("celiac") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("disease") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("middle school") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("amino acid") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bacterial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biofilm") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("larvae") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("naval") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("asparaginases") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cataract") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hepatoxic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("toxic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("vitamin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lumbar") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("viral") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("melatonin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("in vitro ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("in vivo") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("b-cell") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("t-cell") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("interleukin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("zinc") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fluoride") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("stannous") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("macrophage") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antigen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("breast") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("methotrexate") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("perilesional") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("organic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("phosphates") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("organ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("vaccine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharmaceutical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nutrients") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("umbilical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neonate") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biomarker") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sailboat") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("drinking age") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("scrotal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("extrasosseous") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chondroma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("saudi arabia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("embryonic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("saccaharification") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("genetic markers") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("urine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("contaminated") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ground water") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("groundwater") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("outdoor") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("schizo") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hepatocellular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("carcinoma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antibodies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psoriasis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hydroxychloro") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("acrosomal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("mammogram") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("disinfectant") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("germicide") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("tyrosine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("kinase") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("inhibitors") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hemostasis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pollution") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pregnancy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immune") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("salmonella") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("shanghai") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immigrant") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pathogen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ethylene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("phenology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("binge eating") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("xanthan") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ribosomal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chromosomal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("oligonucleotide") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chromosome") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("angiogenesis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bipolar") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("disorder") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("atorvastatin") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("embryo") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytotoxic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("papillomaviruses") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytometry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cervical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cyclophosphamide") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("acupuncture") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nigerian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pancreatitis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pancrea") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("tract infection") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("inflammatory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dermatoglyphic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("corticosteroid") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("xerostomia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuropsychological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("glycemic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurotrophic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("carotene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("prognosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ozone") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("drunk") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("edible") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("proteomic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("mitochondrial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("malignant") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lymphoma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("l-serine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("protease") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("leukocytes") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("phagocytosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sickle") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sexual") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("paternity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("maternity") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemotherapy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharmacogenomics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("mucoadhesive") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("soccer players") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cell membrane") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("benzoyl") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bronchial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gastrointestintal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immune deficiency") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("infectious") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anticoagulate") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sociomaterial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("amsterdam") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("earthquake") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("kidney") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ligniolytic enzyme") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("femur") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("angiogenes") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("serum") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("protein expression") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("health effect") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("carcinogen") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cysteine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharmacoepdemiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("adenine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("guanine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytosene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("thymine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("althlete") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bioavailability") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("green tea") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dentistry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuromuscular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hydroxypropyl") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fibularis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cadaveric") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pickles") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("florescence") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ribosome") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("antidiorrhoeal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chimpanzee") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("newborn") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anesthesiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dermatology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anesthesiolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dermatolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("radiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("radiolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("emergency medicine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("genetics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nuclear medicine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("obstetrics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gynelcolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ophthalmolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("patholog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("rehabilitation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("rehabilitate") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psychiatr") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("oncolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("surgery") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("surgeon") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("urology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytotechnologist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytotechnology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("electroneuro") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("kinesiolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("kinesiotherap") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lactation") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("therapist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("therapy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("perfusion") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("physiotherap") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("polysomnograph") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ambulatory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("audiolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dietitian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dietetic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("therapeutic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ophthalmic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("optometrist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("optometry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("optometr") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("orthopy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("orthopist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("orthop") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("optician") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("dialysis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("electrocardiograph") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("environmental health") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemiologist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemiolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("histolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nurse") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nursery") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("midwife") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("osteopathy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pharmac") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("allopathic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("podiatr") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sonograph") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("veterinar") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("x-ray") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("asthma") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("autism") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("autistic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("influenza") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("avian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("birth defect") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chlamydia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fatigue") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epilepsy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fetal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("herpes") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("genital") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gonorrhea") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("heart disease") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pulmonary") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("traumatic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("zika") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("tuberculosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("scabies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("microcephaly") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("middle east") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("papillomavirus") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("giardiasis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("aesthetic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epistemolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ethics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("metaphysic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("philosophical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ageing") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("human affairs") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("victorian studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("estuaries") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("estuary") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("fungal") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("humanities") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("german studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuroscience") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("utopian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("infants") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("young children") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neurophysiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("aging") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cellulose") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("labor") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("labour") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemistry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("vaccines") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("israel") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("microbiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("lung cancer") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("psychiatry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunohematology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("digestive") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunopathology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("radicalism") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("radical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("caffeine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nursing") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("unicorn") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anthropology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("orthodontics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("orthodont") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immuno") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("eukaryotic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nicotine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("tobacco") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("vascular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("genome") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ecology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("forest science") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("acoustical") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("acoustic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cytogenetic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("respiratory") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("peptidomics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("virology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("adipocyte") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cellular") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("adolescent") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("animal learning") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("social welfare") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ophthalmology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("arthritis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("rheumatism") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nutrafoods") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("morphology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("tissue") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("rhinology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("zoomorphology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biorheology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("toxicology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("nanotoxicology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("anesthesia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hygiene") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("spectrometry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("shellfish") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("analgesia") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bioorganic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("soviet") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("sociolinguistic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("photogrammetry") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemosphere") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("raptor research") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("meridian studies") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("stroke") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("allergy") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("drugs") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("angiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("pancreas") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("neuroendocrinology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("craniofacial") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bioenergetics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bioenergetic") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("traumatology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("periodontology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("methodology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("traumat") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemi") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("globalization") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("electrophysiology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("cardiac ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("ocean") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("health") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("humanit ") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("infections") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("reproductive") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("heart") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("atherosclerosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("thrombosis") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("gerontology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("midwestern") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("bioethics") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("medicine") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("myology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("epidemiological") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("immunology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biolog") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("chemi") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hypertension") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("primatology") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("management") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("metals") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("water") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("hindu") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("muslim") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("christian") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("buddhist") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("missouri") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("peptides") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("clima") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("education") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("behavioral") ]
    dftbr = dftbr[~dftbr.title.str.lower().str.contains("biotech") ]

    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"economics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharmacology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"plant" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"surgery" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"health" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nutrition" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"medical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"molecular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cancer" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"surgical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"medicine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"french" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"political" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"modernism" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"jewish" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nursing" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"wildlife" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"women's studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"philosophy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"english studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"business" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"agriculture" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ecology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"korean studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"zoological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"song-yuan studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"phonetics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"physiotherapy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"geochronometria" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hispanic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pediatric" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psychology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"feminist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ethics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"archaeology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hepatitis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gynecologic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"philology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"paleontology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gender" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ornithology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dental" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurocytology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"petrology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"film" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"history" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"multihandicapped" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"toxicology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"theatre" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuroradiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psycholinguistic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biotelemetry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"christian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hernia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"horticulture" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"photosynthetica" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biological theory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"forestry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"second language" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"seismology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"schizophrenia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"philosophical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psychiatry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"formal logic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"drug" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemists" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"prehistory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"happiness studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gambling" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"economic journal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuroendocrinology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemistry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"soviet" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"food" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"carbonates" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"degradation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"financial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chaucer" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"naturalist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"management control" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"language learning" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"great plains" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"religion" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"geological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"banking" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"jewry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ocean" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"acoustics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"clinical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"botany" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"mammal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"animal science" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anthropological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anthropology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hydrobiological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pain management" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"crime" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"social change" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"african studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fertilizer" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biomed" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cardiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"geotectonics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sports" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"archaeological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"radioanalytical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pedagogy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ornithologists" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biomass" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurophysiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gynecology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"marine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biosphere" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"transplantation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"wood" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"oncology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"heart" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"particle" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"depression" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ultrasonic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gastroenterology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"diversity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"agri " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lipid " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"protein " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"xeno" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nuclear" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"criminology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"metabolism" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"forensic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"allergy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"heredity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"conservation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"atmospheric" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"earth" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"geo" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"aquatic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"husbandry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"rangifier" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"weather" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"potato" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"plasma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"veterinary" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"amphibian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"elife" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"microbiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biometrics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"insect" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"endoscopic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ecosystem" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antiviral" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"armed" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gastroprotective" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"microwave" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"malaysia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"t cells" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"monoclonal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"angiotensin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gastrointestinal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"parasites" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"unemployment" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"television" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"genome" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"soil" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"government" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cultural" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"culture" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"obesity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"uganda" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"b cell" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antioxidant" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"elastin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"collagen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"asia-pacific" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antibacterial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"juice" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lactobacillus" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pelvic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biodegradation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"crude oil" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sediment" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"childbirth" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunoglobulin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"myeloma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"infection" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cardiovascular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"adrenoceptor" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bodyweight" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"squat" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"deadlift" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cardiomyoplasty" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cardio" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"phallus" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"intravesical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"exstrophy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cloacal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fermentation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anger" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"aggression" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"china" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"earlobe" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"estrogen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"atrazine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hydroxytamoxifen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"proliferation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"liver" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cell culture" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"megafauna" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"smoking" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"childhood" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"overwei" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"latissmus" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"myocutaneous" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cervicof" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"selenium" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"maritime" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"agricultural" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sudanese" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"refugee" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"wine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"glycine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"betaine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurodegeneration" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"diabetes" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nitrates" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"myeloid" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"salinity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hindi" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bengali" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"wound" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"postoperative" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nucleus" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"thylakoid" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"celiac" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"disease" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"middle school" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"amino acid" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bacterial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biofilm" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"larvae" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"naval" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"asparaginases" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cataract" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hepatoxic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"toxic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"vitamin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lumbar" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"viral" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"melatonin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"in vitro " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"in vivo" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"b-cell" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"t-cell" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"interleukin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"zinc" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fluoride" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"stannous" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"macrophage" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antigen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"breast" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"methotrexate" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"perilesional" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"organic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"phosphates" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"organ" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"vaccine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharmaceutical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"adolescent" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nutrients" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"umbilical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neonate" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biomarker" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sailboat" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"drinking age" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"scrotal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"extrasosseous" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chondroma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"saudi arabia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"embryonic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"saccaharification" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"genetic markers" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"urine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"contaminated" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ground water" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"groundwater" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"outdoor" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"schizo" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hepatocellular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"carcinoma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antibodies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psoriasis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hydroxychloro" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"acrosomal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"mammogram" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"disinfectant" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"germicide" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"tyrosine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"kinase" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"inhibitors" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hemostasis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"thrombosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"vascular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pollution" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pregnancy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immune" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"salmonella" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"shanghai" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immigrant" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pathogen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ethylene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"phenology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"binge eating" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"xanthan" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ribosomal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chromosomal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"oligonucleotide" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chromosome" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"angiogenesis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bipolar" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"disorder" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"atorvastatin" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"embryo" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytotoxic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"papillomaviruses" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytometry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cervical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cyclophosphamide" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"acupuncture" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nigerian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pancreatitis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pancrea" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"tract infection" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"inflammatory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dermatoglyphic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"corticosteroid" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"xerostomia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuropsychological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"glycemic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurotrophic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"carotene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"prognosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ozone" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"drunk" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"edible" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"proteomic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"mitochondrial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"malignant" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lymphoma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"l-serine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"protease" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"leukocytes" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"phagocytosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sickle" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sexual" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"paternity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"maternity" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemotherapy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharmacogenomics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"mucoadhesive" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"soccer players" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cell membrane" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"benzoyl" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bronchial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gastrointestintal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immune deficiency" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"infectious" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anticoagulate" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sociomaterial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"amsterdam" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"earthquake" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"kidney" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ligniolytic enzyme" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"femur" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"angiogenes" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"serum" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"protein expression" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"health effect" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"carcinogen" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cysteine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharmacoepdemiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"adenine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"guanine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytosene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"thymine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"althlete" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bioavailability" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"green tea" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dentistry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuromuscular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hydroxypropyl" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fibularis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cadaveric" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pickles" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"florescence" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ribosome" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"antidiorrhoeal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chimpanzee" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"newborn" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anesthesiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dermatology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anesthesiolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dermatolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"radiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"radiolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"emergency medicine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"genetics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nuclear medicine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"obstetrics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gynelcolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ophthalmolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"patholog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"rehabilitation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"rehabilitate" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psychiatr" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"oncolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"surgery" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"surgeon" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"urology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytotechnologist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytotechnology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"electroneuro" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"kinesiolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"kinesiotherap" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lactation" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"therapist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"therapy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"perfusion" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"physiotherap" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"polysomnograph" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"respiratory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ambulatory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anesthesia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"audiolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dietitian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dietetic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"therapeutic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ophthalmic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"optometrist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"optometry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"optometr" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"orthopy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"orthopist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"orthop" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"optician" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"humanities" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"dialysis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"electrocardiograph" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"environmental health" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemiologist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemiolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"histolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nurse" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nursery" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"midwife" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"osteopathy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pharmac" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"allopathic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"podiatr" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sonograph" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"veterinar" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"x-ray" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"arthritis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"asthma" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"autism" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"autistic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"influenza" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"avian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"birth defect" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chlamydia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fatigue" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epilepsy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fetal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"herpes" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"genital" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gonorrhea" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"heart disease" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pulmonary" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"stroke" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"traumatic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"zika" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"tuberculosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"scabies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"microcephaly" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"middle east" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"papillomavirus" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"giardiasis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"aesthetic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epistemolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ethics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"metaphysic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"philosophical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ageing" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"human affairs" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"victorian studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"estuaries" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"estuary" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"fungal" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"humanities" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"german studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuroscience" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"utopian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"infants" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"young children" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neurophysiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"aging" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cellulose" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"labor" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"labour" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemistry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"vaccines" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"israel" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"microbiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"lung cancer" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"psychiatry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunohematology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"digestive" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunopathology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"radicalism" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"radical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"caffeine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nursing" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"unicorn" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anthropology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"orthodontics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"orthodont" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immuno" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"eukaryotic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nicotine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"tobacco" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"vascular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"genome" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ecology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"forest science" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"acoustical" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"acoustic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cytogenetic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"respiratory" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"peptidomics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"virology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"adipocyte" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cellular" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"adolescent" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"animal learning" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"social welfare" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ophthalmology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"arthritis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"rheumatism" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nutrafoods" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"morphology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"tissue" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"rhinology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"zoomorphology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biorheology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"toxicology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"nanotoxicology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"anesthesia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hygiene" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"spectrometry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"shellfish" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"analgesia" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bioorganic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"soviet" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"sociolinguistic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"photogrammetry" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemosphere" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"raptor research" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"meridian studies" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"stroke" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"allergy" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"drugs" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"angiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"pancreas" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"neuroendocrinology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"craniofacial" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bioenergetics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bioenergetic" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"traumatology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"periodontology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"methodology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"traumat" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemi" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"globalization" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"electrophysiology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"cardiac " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"ocean" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"health" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"humanit " in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"infections" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"reproductive" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"heart" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"atherosclerosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"thrombosis" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"gerontology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"midwestern" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"bioethics" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"medicine" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"myology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"epidemiological" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"immunology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biolog" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"chemi" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hypertension" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"primatology" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"management" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"metals" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"water" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"hindu" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"muslim" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"christian" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"buddhist" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"missouri" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"peptides" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"clima" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"education" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"behavioral" in lowerCaseElements(x))]
    dftbr = dftbr[~dftbr.entities.apply(lambda x:"biotech" in lowerCaseElements(x))]
    length = dftbr.shape[0]
    # dftbr = dftbr[[ 'id', 'title', 's2Url', 'totalCites2' ]]
    print("dftbr shape is ", dftbr.shape)
    return dftbr

In [0]:
def remove_nonenglish(df, jSet, vSet):
  # Gets list of English Journals
  listOfEnglishJournals = []
  identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
  count = 0
  for i in jSet:
    lang = identifier.classify(str(i))
    langstr = str(lang)
    if langstr.find("en") != -1 and lang[1] > 0.5:
      listOfEnglishJournals.append(i)
  
  # Gets List of English Venues
  listOfEnglishVenues = []
  count = 0
  for i in vSet:
    lang = identifier.classify(str(i))
    langstr = str(lang)
    if langstr.find("en") != -1 and lang[1] > 0.5:
      listOfEnglishVenues.append(i)

  filter1 = df_refined['journalName'].isin(listOfEnglishJournals)
  filter2 = df_refined['venue'].isin(listOfEnglishVenues)

  dfFinal = df_refined[df_refined['journalName'].isin(listOfEnglishJournals) | df_refined['venue'].isin(listOfEnglishVenues)]

  return dfFinal


# Testing/Prototyping (Ignore)

In [0]:
url_template = 'https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2019-11-01/s2-corpus-{:03}.gz'
file_template = 'json{:03}.gz'
json_template = 'json{:03}'

In [0]:
#0, 1, 2, 3
i = 55
mylist = dlandopen(url_template.format(i), file_template.format(i), json_template.format(i) )

In [0]:
mylist

['{"entities":[],"journalVolume":"","journalPages":"","pmid":"","year":1980,"outCitations":[],"s2Url":"https://semanticscholar.org/paper/64ecbd6c76993ba2257df1197db7d07f7449313f","s2PdfUrl":"","id":"64ecbd6c76993ba2257df1197db7d07f7449313f","authors":[{"name":"大石 昂","ids":["135576646"]}],"journalName":"","paperAbstract":"","inCitations":[],"pdfUrls":[],"title":"269 回転図形と菱形の再生困難性(発達7,発達)","doi":"10.20587/pamjaep.22.0_294","sources":[],"doiUrl":"https://doi.org/10.20587/pamjaep.22.0_294","venue":""}\n',
 '{"entities":[],"journalVolume":"","journalPages":"","pmid":"","year":1973,"outCitations":[],"s2Url":"https://semanticscholar.org/paper/5cbb4a7811b6e9a0715aefe10ff8af79ffd71ab7","s2PdfUrl":"","id":"5cbb4a7811b6e9a0715aefe10ff8af79ffd71ab7","authors":[{"name":"Edouard Fabre","ids":["91318806"]},{"name":"Rameshwar N. Bhargava","ids":["145191430"]}],"journalName":"","paperAbstract":"","inCitations":[],"pdfUrls":[],"title":"Thermally stimulated current (T.S.C.) measurements and their correla

In [0]:
df = firstDataframe(mylist, 0, len(mylist))
#df.to_json("/content/gdrive/My Drive/text2doc_data/test_ignore/test-{:03}.json".format(i))

In [0]:
list = []
for i in final_df['pmid']:
  list.append(i)


In [0]:
list

['16043399',
 '26781345',
 '24435717',
 '30827415',
 '19366635',
 '26724916',
 '23108793',
 '29085016',
 '25650214',
 '22831821',
 '27940289',
 '14709124',
 '27399951',
 '19932533',
 '20810086',
 '28679698',
 '23701977',
 '27715419',
 '25699249',
 '18340524',
 '24960014',
 '526256',
 '15984693',
 '23717558',
 '28636649',
 '24285083',
 '17404857',
 '7749567',
 '26702673',
 '11320043',
 '1691264',
 '15536471',
 '20603684',
 '27277803',
 '30965554',
 '29846846',
 '20967303',
 '29755615',
 '28844195',
 '27648233',
 '26887969',
 '15675461',
 '15319192',
 '20140088',
 '22173361',
 '1351768',
 '23493011',
 '31032026',
 '18421004',
 '20147418',
 '17709436',
 '21279455',
 '16359264',
 '14747637',
 '30102186',
 '15451939',
 '30675467',
 '2546095',
 '17702967',
 '20840610',
 '25267796',
 '19921512',
 '21174530',
 '26564792',
 '25089325',
 '21922281',
 '18559815',
 '26668616',
 '18519135',
 '17982656',
 '10225985',
 '25324097',
 '30730582',
 '21216598',
 '30856626',
 '23264806',
 '19665774',
 '253

In [0]:
df_refined = medicalRefinement(df)
i = 55
print("REFINED")
journalSet = set()
venueSet = set()
print("SETS INIT")
journalSet = journalSet|set(df_refined['journalName'])
venueSet = journalSet|set(df_refined['venue'])
print("SETS CREATED")
final_df = remove_nonenglish(df_refined, journalSet, venueSet)
final_df.to_json("/content/gdrive/My Drive/text2doc_data/med_data/med-{:03}.json".format(i))

REFINED
SETS INIT
SETS CREATED


NameError: ignored

In [0]:
df_refined.shape

(112419, 21)

In [0]:
final_df = remove_nonenglish(df_refined, journalSet, venueSet)


In [0]:
df

,entities,journalVolume,journalPages,pmid,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,Sources3
0,[],,,,1980.0,[],https://semanticscholar.org/paper/64ecbd6c7699...,,64ecbd6c76993ba2257df1197db7d07f7449313f,"[{'name': '大石 昂', 'ids': ['135576646']}]",,,[],[],"269 回転図形と菱形の再生困難性(発達7,発達)",10.20587/pamjaep.22.0_294,[],https://doi.org/10.20587/pamjaep.22.0_294,,[]
1,[],,,,1973.0,[],https://semanticscholar.org/paper/5cbb4a7811b6...,,5cbb4a7811b6e9a0715aefe10ff8af79ffd71ab7,"[{'name': 'Edouard Fabre', 'ids': ['91318806']...",,,[],[],Thermally stimulated current (T.S.C.) measurem...,,[],,,[]
2,[],116,2184-2203,16043399,2005.0,"[0bb6eee3b9a7d3f99f6f61c82b9458395e20c650, d1d...",https://semanticscholar.org/paper/590456d006bb...,,590456d006bbd327babb2402746c381dbbe81d03,"[{'name': 'Donna Coch', 'ids': ['1926564']}, {...",Clinical Neurophysiology,OBJECTIVE\nThis developmental study was design...,"[b2b94350d27321fb1047ea776a6713521ba570c6, 119...",[https://bdl.uoregon.edu/files/2012/09/Coch-Cl...,Auditory and visual refractory period effects ...,10.1016/j.clinph.2005.06.005,[Medline],https://doi.org/10.1016/j.clinph.2005.06.005,Clinical Neurophysiology,['Medline']
3,[],70 2 Suppl,\n S20-32\n,10966007,2000.0,[],https://semanticscholar.org/paper/b6374a068a5f...,,b6374a068a5ff9ab4074eb53913de0ba14e0eae8,"[{'name': 'George John Despotis', 'ids': ['505...",The Annals of thoracic surgery,Patients undergoing cardiac surgery with cardi...,"[d33812eeb35b3d7651ed6f4198871c466cd63c42, 956...",[],Management approaches to platelet-related micr...,10.1016/S0003-4975(00)01604-0,[Medline],https://doi.org/10.1016/S0003-4975%2800%2901604-0,The Annals of thoracic surgery,['Medline']
4,[],,,,2003.0,[],https://semanticscholar.org/paper/f741ca502dfc...,http://pdfs.semanticscholar.org/f741/ca502dfcc...,f741ca502dfcc21599022efe326fcd69724e5e4e,"[{'name': 'Kathy Konicek', 'ids': ['70506139']...",,Electronic reserves help registered campus use...,"[905fd5a2e1448b9b47d456cc4b5e0f83e6802a4f, 92c...",[http://www.uvm.edu/~bnelson/computer/accessib...,Electronic reserves: the promise and challenge...,10.1108/07378830310467445,[],https://doi.org/10.1108/07378830310467445,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996184,[],,,,2007.0,[],https://semanticscholar.org/paper/285bc3216d0a...,,285bc3216d0aae217115be57ad6ab04682048766,"[{'name': 'Sonja Brentjes', 'ids': ['47950428']}]",,This French translation of Hernandez’s origina...,[],[],"Histoire de la pensée en terre d'Islam. Paris,...",,[],,,[]
996185,[],1966,,,1993.0,"[4ae72c0167138823120ee4d033966d52a30331ef, ded...",https://semanticscholar.org/paper/a31ccd38d478...,,a31ccd38d47874dff12ee306b33850c34f770443,"[{'name': 'Stuart J. Flockton', 'ids': ['20859...",,Adaptive infmite impulse response filters requ...,"[402b62d3642f414ad9356df49229be17fb19b02b, da6...",[http://www.proxy.rhbnc.ac.uk/~uhap016/publica...,Application of genetic algorithms to infinite-...,10.1117/12.152641,[],https://doi.org/10.1117/12.152641,"Defense, Security, and Sensing",[]
996186,[],,,,2008.0,[],https://semanticscholar.org/paper/8c77735a4499...,,8c77735a4499117b3439e857c56b7c9ee19c940a,"[{'name': 'Wang Ling', 'ids': ['144365797']}]",,Automobile phosphate coating was one chemical ...,[],[],Research of Automobile Phosphating Wastewater ...,,[],,,[]
996187,[],,,,1989.0,[],https://semanticscholar.org/paper/dee1724e61a2...,,dee1724e61a2e2959b56594e2e7819bfa21583b7,"[{'name': '강희천', 'ids': ['104921914']}]",,,[],[],기독교교육과 언어의 기능,,[],,,[]


In [0]:
final_df

,entities,journalVolume,journalPages,pmid,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,strsources
2,[],116,2184-2203,16043399,2005.0,"[0bb6eee3b9a7d3f99f6f61c82b9458395e20c650, d1d...",https://semanticscholar.org/paper/590456d006bb...,,590456d006bbd327babb2402746c381dbbe81d03,"[{'name': 'Donna Coch', 'ids': ['1926564']}, {...",Clinical Neurophysiology,OBJECTIVE\nThis developmental study was design...,"[b2b94350d27321fb1047ea776a6713521ba570c6, 119...",[https://bdl.uoregon.edu/files/2012/09/Coch-Cl...,Auditory and visual refractory period effects ...,10.1016/j.clinph.2005.06.005,[Medline],https://doi.org/10.1016/j.clinph.2005.06.005,Clinical Neurophysiology,['Medline']
25,[],89 1059,\n 20150493\n,26781345,2016.0,"[4d8b9fe5bcbf115c3db2790488b1970ad22a53dc, 72b...",https://semanticscholar.org/paper/c8fa150c9e6a...,,c8fa150c9e6ae83687d2c807bd7ca9966d943678,"[{'name': 'James W Mackay', 'ids': ['50484327'...",The British journal of radiology,OBJECTIVE\nTo determine whether differences in...,[bdfc6cece0fe624c27b9cf02f2602e78ad304358],[],Ultrashort TE evaluation of the osteochondral ...,10.1259/bjr.20150493,[Medline],https://doi.org/10.1259/bjr.20150493,The British journal of radiology,['Medline']
28,[],39,706-711,24435717,2013.0,"[910812f6ffd6204f54748a2dcb737ffced3b1907, aa4...",https://semanticscholar.org/paper/37a37ba06069...,,37a37ba060696253c85a30d29455b9c76e88c987,"[{'name': 'Porschea Lewis', 'ids': ['49841476'...",Journal of Community Health,Little is known about disparities in preventiv...,"[46d6197336a3949b7ad4bcbf4301bb272388cb01, 018...",[],Racial Disparities at the Point of Care for Ur...,10.1007/s10900-013-9815-5,[Medline],https://doi.org/10.1007/s10900-013-9815-5,Journal of Community Health,['Medline']
40,[],7,480 - 482.e7,30827415,2019.0,"[b439f74a55ad07da6207629bff5cd6dfdd7a32fd, 3ed...",https://semanticscholar.org/paper/27538cb63f1e...,,27538cb63f1ea04067aba9fa59853f64bae869da,"[{'name': 'Ravindran Caspa Gokulan', 'ids': ['...",,Epidemiologic studies have identified gastroes...,[],[],Gastroesophageal Reflux Induces Protein Adduct...,10.1016/j.jcmgh.2018.10.017,[Medline],https://doi.org/10.1016/j.jcmgh.2018.10.017,Cellular and molecular gastroenterology and he...,['Medline']
49,[],46 8,\n 517-30\n,19366635,2009.0,"[a7f8d107fc3e8a26f44df956145a878a8bdb18b2, bab...",https://semanticscholar.org/paper/4f92c7596682...,,4f92c759668246d8f07c82893d525a4655660a8d,"[{'name': 'Dong-Xiu Zhang', 'ids': ['6399368']...",Fungal genetics and biology : FG & B,Epichloë endophytes (holomorphic Epichloë spp....,"[3fd852caecf50d132e18fba4a78ac15e8e16d3df, 5ac...",[],Coregulated expression of loline alkaloid-bios...,10.1016/j.fgb.2009.03.010,[Medline],https://doi.org/10.1016/j.fgb.2009.03.010,Fungal genetics and biology : FG & B,['Medline']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995952,[],23 11,\n 2983-9\n,22689178,2012.0,"[b4d53f1eb2198f45ada833b80b021f2b91746b1e, 68c...",https://semanticscholar.org/paper/f1835d06b661...,http://pdfs.semanticscholar.org/f183/5d06b661a...,f1835d06b661a263cca63430d67a0a82682eb36a,"[{'name': 'Junbo Wang', 'ids': ['5482750']}, {...",Annals of oncology : official journal of the E...,BACKGROUND\nMost cancers are due to modifiable...,"[85ada8a3fd7dd4ae08cd1818ff7d54355944d02b, d86...",[https://oup.silverchair-cdn.com/oup/backfile/...,Attributable causes of cancer in China.,10.1093/annonc/mds139,[Medline],https://doi.org/10.1093/annonc%2Fmds139,Annals of oncology : official journal of the E...,['Medline']
995996,[],251 1,\n 33-35\n,28621595,2017.0,"[96a953b1b7cf3b0af424e3ffe528640bc954ca9b, ef7...",https://semanticscholar.org/paper/b0a28a661ec1...,,b0a28a661ec14ff5b52ac514b5509cf25c4a0f95,"[{'name': 'Bill Hartman', 'ids': ['35147530']}...",Journal of the American Veterinary Medical Ass...,33 Educating clients and veterinary students c...,[],[],Optimizing client and student learning from th...,10.

In [0]:
finalJournalSet = journalSet|set(final_df['journalName'])
finalVenueSet = journalSet|set(final_df['venue'])

In [0]:
finalJournalSet

{'',
 '2016 IEEE International Conference on Systems, Man, and Cybernetics (SMC)',
 '2017 IEEE Innovative Smart Grid Technologies - Asia (ISGT-Asia)',
 'International Conference on Oxide Materials for Electronic Engineering - fabrication, properties and applications (OMEE-2014)',
 '2006 29th International Spring Seminar on Electronics Technology',
 '2017 IEEE International Conference on Smart Computing (SMARTCOMP)',
 'Conference Record of the Thirty-Third Asilomar Conference on Signals, Systems, and Computers (Cat. No.CH37020)',
 'Proceedings of the IEEE 1995 Custom Integrated Circuits Conference',
 'IZA Journal of Migration',
 '2017 IEEE Green Energy and Smart Systems Conference (IGESSC)',
 '2018 25th Saint Petersburg International Conference on Integrated Navigation Systems (ICINS)',
 '2014 International Conference on Circuits, Power and Computing Technologies [ICCPCT-2014]',
 '2016 IEEE Intelligent Vehicles Symposium (IV)',
 'Applied Mathematics-A Journal of Chinese Universities',
 

In [0]:
final_df

,entities,journalVolume,journalPages,pmid,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,Sources3,totalCites2
103,[],,213-218,,2015.0,"[9c7cb6b3aa82b3765eea02b0de67d084a76362e5, 551...",https://semanticscholar.org/paper/a9dbb9d8bf1b...,,a9dbb9d8bf1b29bde43534f564cf5f504da024dc,"[{'name': 'J. David Stevens', 'ids': ['1444237...",2015 IEEE Electric Ship Technologies Symposium...,The design and control of electric warship pow...,"[7fafb2178618dc485bf92b5279b82dbb5a216d41, db1...",[https://www.usna.edu/Users/ee/opila/files/EST...,Operational vignette-based electric warship lo...,10.1109/ESTS.2015.7157890,[],https://doi.org/10.1109/ESTS.2015.7157890,2015 IEEE Electric Ship Technologies Symposium...,[],"[9c7cb6b3aa82b3765eea02b0de67d084a76362e5, 551..."
183,[],,2635-2637,,2018.0,"[5d46a22318bc0e70f3b9ec0fbbb1017e3ead89ab, 69b...",https://semanticscholar.org/paper/04c5e4d1f347...,,04c5e4d1f347d78afddec7539837005145d8aa6e,"[{'name': 'Robby Peibst', 'ids': ['2034407']},...",2018 IEEE 7th World Conference on Photovoltaic...,We present a novel cell concept that combines ...,[],[],p+/n+ polysilicon-on-oxide tunneling junctions...,10.1109/pvsc.2018.8548032,[],https://doi.org/10.1109/pvsc.2018.8548032,2018 IEEE 7th World Conference on Photovoltaic...,[],"[5d46a22318bc0e70f3b9ec0fbbb1017e3ead89ab, 69b..."
386,[],,1269-1274,,2015.0,"[1f4f171da0827b512d63e0187dc33e1b2fb624b1, aad...",https://semanticscholar.org/paper/a7acf49e2ff4...,,a7acf49e2ff4c8a824c2274370d9f752445ace7f,"[{'name': 'Ilan Aharon', 'ids': ['35194475']},...",2015 International Conference on Renewable Ene...,"In this paper, a design of linear robust contr...",[],[],Robust UDE controller for energy storage appli...,10.1109/ICRERA.2015.7418612,[],https://doi.org/10.1109/ICRERA.2015.7418612,2015 International Conference on Renewable Ene...,[],"[1f4f171da0827b512d63e0187dc33e1b2fb624b1, aad..."
440,[],6,1-7,,2019.0,"[98e4e80334a18c2184ba2190061e7ea7409af6df, 687...",https://semanticscholar.org/paper/5782b0f84242...,,5782b0f84242ebc6591bdc2cb3c366c0886717fb,"[{'name': 'Ahmadreza Shirvani Dastgerdi', 'ids...","City, Territory and Architecture",City brand is a valuable asset that improves t...,[],[https://cityterritoryarchitecture.springerope...,Strengthening the city’s reputation in the age...,10.1186/s40410-019-0101-4,[],https://doi.org/10.1186/s40410-019-0101-4,"City, Territory and Architecture",[],"[98e4e80334a18c2184ba2190061e7ea7409af6df, 687..."
589,[],,380-383,,2014.0,"[51e4c22e5c1fb8cf5572d07a0bec4011f3c8d9eb, f7f...",https://semanticscholar.org/paper/743fa37c4194...,,743fa37c4194ae87c00f174ee43f88924ab63ae5,"[{'name': 'Rami Al-Dalky', 'ids': ['3019216']}...",2014 9th International Symposium on Communicat...,Network's speed continues to increase at a hig...,[],[https://doi.org/10.1109/CSNDSP.2014.6923858],Framework for a NetFPGA-based Snort NIDS,10.1109/CSNDSP.2014.6923858,[DBLP],https://doi.org/10.1109/CSNDSP.2014.6923858,2014 9th International Symposium on Communicat...,['DBLP'],"[51e4c22e5c1fb8cf5572d07a0bec4011f3c8d9eb, f7f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995576,[],,613-618,,2013.0,"[adc326a83bafc38443e5c9b09bc5209249b72cbc, 6f6...",https://semanticscholar.org/paper/b5325c07cef5...,,b5325c07cef56735ed2d807f5676bc780c3d949f,"[{'name': 'Jen-Chun Chang', 'ids': ['39969722'...",2013 IEEE 10th International Conference on Ubi...,We study hash-based identification protocols i...,[ad22df31eb9d51187255b771d660c01e584fbb61],[http://www.iis.sinica.edu.tw/page/jise/2016/2...,On Constant-Time-Identification and Privacy-Pr...,10.1109/UIC-ATC.2013.92,[DBLP],https://doi.org/10.1109/UIC-ATC.2013.92,2013 IEEE 10th International Conference on Ubi...,['DBLP'],"[adc326a83bafc38443e5c9b09bc5209249b72cbc, 6f6..."
995853,[],95,215-220,,2009.0,[],https://semanticscholar.org/paper/9ad0db010d1a...,,9ad0db010d1a5309de79e279fd267cc4dbc02592,"[{'name': 'Jana Výchopňová'

In [0]:
final_df.head(1000)

,entities,journalVolume,journalPages,pmid,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,Sources3,totalCites2
103,[],,213-218,,2015.0,"[9c7cb6b3aa82b3765eea02b0de67d084a76362e5, 551...",https://semanticscholar.org/paper/a9dbb9d8bf1b...,,a9dbb9d8bf1b29bde43534f564cf5f504da024dc,"[{'name': 'J. David Stevens', 'ids': ['1444237...",2015 IEEE Electric Ship Technologies Symposium...,The design and control of electric warship pow...,"[7fafb2178618dc485bf92b5279b82dbb5a216d41, db1...",[https://www.usna.edu/Users/ee/opila/files/EST...,Operational vignette-based electric warship lo...,10.1109/ESTS.2015.7157890,[],https://doi.org/10.1109/ESTS.2015.7157890,2015 IEEE Electric Ship Technologies Symposium...,[],"[9c7cb6b3aa82b3765eea02b0de67d084a76362e5, 551..."
183,[],,2635-2637,,2018.0,"[5d46a22318bc0e70f3b9ec0fbbb1017e3ead89ab, 69b...",https://semanticscholar.org/paper/04c5e4d1f347...,,04c5e4d1f347d78afddec7539837005145d8aa6e,"[{'name': 'Robby Peibst', 'ids': ['2034407']},...",2018 IEEE 7th World Conference on Photovoltaic...,We present a novel cell concept that combines ...,[],[],p+/n+ polysilicon-on-oxide tunneling junctions...,10.1109/pvsc.2018.8548032,[],https://doi.org/10.1109/pvsc.2018.8548032,2018 IEEE 7th World Conference on Photovoltaic...,[],"[5d46a22318bc0e70f3b9ec0fbbb1017e3ead89ab, 69b..."
386,[],,1269-1274,,2015.0,"[1f4f171da0827b512d63e0187dc33e1b2fb624b1, aad...",https://semanticscholar.org/paper/a7acf49e2ff4...,,a7acf49e2ff4c8a824c2274370d9f752445ace7f,"[{'name': 'Ilan Aharon', 'ids': ['35194475']},...",2015 International Conference on Renewable Ene...,"In this paper, a design of linear robust contr...",[],[],Robust UDE controller for energy storage appli...,10.1109/ICRERA.2015.7418612,[],https://doi.org/10.1109/ICRERA.2015.7418612,2015 International Conference on Renewable Ene...,[],"[1f4f171da0827b512d63e0187dc33e1b2fb624b1, aad..."
440,[],6,1-7,,2019.0,"[98e4e80334a18c2184ba2190061e7ea7409af6df, 687...",https://semanticscholar.org/paper/5782b0f84242...,,5782b0f84242ebc6591bdc2cb3c366c0886717fb,"[{'name': 'Ahmadreza Shirvani Dastgerdi', 'ids...","City, Territory and Architecture",City brand is a valuable asset that improves t...,[],[https://cityterritoryarchitecture.springerope...,Strengthening the city’s reputation in the age...,10.1186/s40410-019-0101-4,[],https://doi.org/10.1186/s40410-019-0101-4,"City, Territory and Architecture",[],"[98e4e80334a18c2184ba2190061e7ea7409af6df, 687..."
589,[],,380-383,,2014.0,"[51e4c22e5c1fb8cf5572d07a0bec4011f3c8d9eb, f7f...",https://semanticscholar.org/paper/743fa37c4194...,,743fa37c4194ae87c00f174ee43f88924ab63ae5,"[{'name': 'Rami Al-Dalky', 'ids': ['3019216']}...",2014 9th International Symposium on Communicat...,Network's speed continues to increase at a hig...,[],[https://doi.org/10.1109/CSNDSP.2014.6923858],Framework for a NetFPGA-based Snort NIDS,10.1109/CSNDSP.2014.6923858,[DBLP],https://doi.org/10.1109/CSNDSP.2014.6923858,2014 9th International Symposium on Communicat...,['DBLP'],"[51e4c22e5c1fb8cf5572d07a0bec4011f3c8d9eb, f7f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92018,[],2899,,,1996.0,[d357ae13352fc8e9f4b292642d59322044b51fbf],https://semanticscholar.org/paper/7f55b90bcde9...,,7f55b90bcde961a3f663bba936267b62e809e1f9,"[{'name': 'Fengzhou Fang', 'ids': ['50346846']...",,Laser measuring is one of the main methods in ...,[],[],Principle of AOM laser heterodyne measuring an...,10.1117/12.253094,[],https://doi.org/10.1117/12.253094,Other Conferences,[],[d357ae13352fc8e9f4b292642d59322044b51fbf]
92405,[],45,68-72,,1997.0,[],https://semanticscholar.org/paper/b1f3e3dc6e14...,,b1f3e3dc6e14730f3f5acb5202803a11574d8733,"[{'name': 'Ray L. Frost', 'ids': ['3141563']},...",Clays and Clay Minerals,The Raman spectra of a tubular halloysite orig...,"[7213f9c8f7ba00a0ec674b41094f7d576f43f580, a0d...",[],Raman Microprobe Spectroscopy of Halloysite,10.1346/CCMN.1997.

In [0]:
s = "hello"
if s.find("g") != -1:
  print(s)

In [0]:
from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
lang = identifier.classify("Experimental Astronomy")
print(lang)
lang[1]

('it', 0.46741224233349316)


0.46741224233349316

In [0]:
#Prints all non-english journals
listOfNonEnglishJournals = []
from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
count = 0
for i in initialJournalSet:
  lang = identifier.classify(str(i))
  langstr = str(lang)
  if langstr.find("en") == -1 and lang[1] > 0.5:
    listOfNonEnglishJournals.append(i)
    print(i)
    count += 1
print(count)

Israel Studies
2017 IEEE Innovative Smart Grid Technologies - Asia (ISGT-Asia)
Advances in gerontology = Uspekhi gerontologii
Tijdschrift voor VerpleeghuisGeneeskunde
2016 IEEE Intelligent Vehicles Symposium (IV)
Phlebologie
Medecine et sante tropicales
Neural Networks
Fasciculi Mathematici
Der Urologe, Ausgabe A
Zeitschrift fur die gesamte innere Medizin und ihre Grenzgebiete
Zeitschrift für die gesamte experimentelle Medizin einschließlich experimentelle Chirurgie
Chirurgie pediatrique
Advance data
Prensa medica argentina
Zeitschrift fur Naturforschung. Teil B, Chemie, Biochemie, Biophysik, Biologie und verwandte Gebiete
Schizophrenia bulletin
Bijzijn XL
Annali di ostetricia, ginecologia, medicina perinatale
Archivo espanol de morfologia
Canadian journal of applied sport sciences. Journal canadien des sciences appliquees au sport
Zhonghua xue ye xue za zhi = Zhonghua xueyexue zazhi
Fortschritte der Kieferorthopadie
Zeitschrift für Unternehmensgeschichte
Giornale italiano di dermatolo

In [0]:
listOfNonEnglishJournals

['Israel Studies',
 '2017 IEEE Innovative Smart Grid Technologies - Asia (ISGT-Asia)',
 'Advances in gerontology = Uspekhi gerontologii',
 'Tijdschrift voor VerpleeghuisGeneeskunde',
 '2016 IEEE Intelligent Vehicles Symposium (IV)',
 'Phlebologie',
 'Medecine et sante tropicales',
 'Neural Networks',
 'Fasciculi Mathematici',
 'Der Urologe, Ausgabe A',
 'Zeitschrift fur die gesamte innere Medizin und ihre Grenzgebiete',
 'Zeitschrift für die gesamte experimentelle Medizin einschließlich experimentelle Chirurgie',
 'Chirurgie pediatrique',
 'Advance data',
 'Prensa medica argentina',
 'Zeitschrift fur Naturforschung. Teil B, Chemie, Biochemie, Biophysik, Biologie und verwandte Gebiete',
 'Schizophrenia bulletin',
 'Bijzijn XL',
 'Annali di ostetricia, ginecologia, medicina perinatale',
 'Archivo espanol de morfologia',
 'Canadian journal of applied sport sciences. Journal canadien des sciences appliquees au sport',
 'Zhonghua xue ye xue za zhi = Zhonghua xueyexue zazhi',
 'Fortschritte 

In [0]:
import langid
def isEnglish(row):
  identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
  lang = identifier.classify(str(row.journalName))
  langstr = str(lang)
  if langstr.find("en") == -1 and lang[1] > 0.5:
    return True
  else:
    return False

def lang(row):
  try:
    if isEnglish(str(row["journalName"])) == True:
      print("worked")
      return row["journalName"]
    else:
      print("worked")
      return "NOT_ENGLISH"
  except:
    print("json error", row["id"])
    return row["journalName"]

In [0]:
dfEnglish = df[~df['journalName'].isin(listOfNonEnglishJournals)]

In [0]:
dfNonEnglishOnly

NameError: ignored

In [0]:
dfEnglish

In [0]:
i = 12
dfEnglish.to_csv("/content/gdrive/My Drive/text2doc_data/test_ignore/test-{:03}.csv".format(i))

#Final Code For Cleaning + Uploading all Open Corpus




In [0]:
url_template = 'https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2019-11-01/s2-corpus-{:03}.gz'
file_template = 'json{:03}.gz'
json_template = 'json{:03}'
i = 151

with tqdm(total = 100) as pbar:
  while i <= 178:
    print ("Starting", i)
    pbar.update()
    mylist = dlandopen(url_template.format(i), file_template.format(i), json_template.format(i) )
    print("File Unzipped " + str(i))
    pbar.update()
    df = firstDataframe(mylist, 0, len(mylist))
    df_refined = dfRefinement(df)
    print("Initial Refinement " + str(i))
    pbar.update()
    journalSet = set()
    venueSet = set()

    journalSet = journalSet|set(df_refined['journalName'])
    venueSet = journalSet|set(df_refined['venue'])
    print("Sets Created " + str(i))
    pbar.update()

    final_df = remove_nonenglish(df_refined, journalSet, venueSet)
    print("Nonenglish Removed, Final Data Created " + str(i))
    pbar.update()

    final_df.to_json("/content/gdrive/My Drive/text2doc_data/s2-cleaned-corpus-{:03}.json".format(i), orient='index')
    pbar.update()

    i += 1  

  0%|          | 0/100 [00:00<?, ?it/s]

Starting 151


  2%|▏         | 2/100 [00:46<37:58, 23.25s/it]

File Unzipped 151


  3%|▎         | 3/100 [17:20<8:28:14, 314.37s/it]

...

165it [9:14:12, 466.01s/it]

Initial Refinement 178
Sets Created 178


167it [9:14:41, 330.47s/it]

Nonenglish Removed, Final Data Created 178


168it [9:14:41, 231.41s/it]


In [0]:
final_df

NameError: ignored